# P4

## Récupération des données

In [263]:
import pandas as pd
import numpy as np

ALLFILES = True
SPARSE = False
CT_DIR_DATA = 'C:\Work\OpenClassrooms\data\P4/'

filename = '2016_'
# Récupération du 1er fichier
dataraw = pd.read_csv(CT_DIR_DATA + filename + '01_ready.csv', sep="\t", encoding='utf-8')
print(dataraw.shape)

if ALLFILES:
    # Concaténation de toutes les données
    for i in range(2, 13):
        n = CT_DIR_DATA + filename + '%0*d' % (2, i) + '_ready.csv'
        #print('reading', n, '...')
        f = pd.read_csv(n, sep="\t", encoding='utf-8')
        print(n, f.shape)
        dataraw = dataraw.append(f)
        print(dataraw.shape)


(433298, 13)
reading 2016_02_ready.csv ...
2016_02_ready.csv (416097, 12)
(849395, 13)
reading 2016_03_ready.csv ...
2016_03_ready.csv (473286, 12)
(1322681, 13)
reading 2016_04_ready.csv ...
2016_04_ready.csv (474753, 12)
(1797434, 13)
reading 2016_05_ready.csv ...
2016_05_ready.csv (475499, 12)
(2272933, 13)
reading 2016_06_ready.csv ...
2016_06_ready.csv (480916, 12)
(2753849, 13)
reading 2016_07_ready.csv ...
2016_07_ready.csv (491198, 12)
(3245047, 13)
reading 2016_08_ready.csv ...
2016_08_ready.csv (489965, 12)
(3735012, 13)
reading 2016_09_ready.csv ...
2016_09_ready.csv (452558, 12)
(4187570, 13)
reading 2016_10_ready.csv ...
2016_10_ready.csv (467486, 12)
(4655056, 13)
reading 2016_11_ready.csv ...
2016_11_ready.csv (448911, 12)
(5103967, 13)
reading 2016_12_ready.csv ...
2016_12_ready.csv (452229, 12)
(5556196, 13)


## Préparation au calcul

In [264]:

if ALLFILES:
    dfCARRIER = dataraw
else:
    dfCARRIER = dataraw.sample(10000)
    #dfCARRIER = dataraw[dataraw.UNIQUE_CARRIER=='AA']
dfCARRIER.shape
if not pd.api.types.is_int64_dtype(dfCARRIER.UNIQUE_CARRIER.dtypes):
    print(set(dfCARRIER['UNIQUE_CARRIER']))
dataraw.UNIQUE_CARRIER.dtypes

{'VX', 'B6', 'AA', 'NK', 'OO', 'F9', 'AS', 'WN', 'EV', 'DL', 'UA', 'HA'}


dtype('O')

In [265]:
# On veut uniquement des valeurs numériques
from sklearn.preprocessing import LabelEncoder
if SPARSE:
    # Uniquement des valeurs numériques pour OneHotEncoder
    if not pd.api.types.is_int64_dtype(dfCARRIER.UNIQUE_CARRIER.dtypes):
        lbl = LabelEncoder().fit(dfCARRIER.UNIQUE_CARRIER)
        dfCARRIER['UNIQUE_CARRIER'] = lbl.transform(dfCARRIER.UNIQUE_CARRIER)
        print(set(dfCARRIER['UNIQUE_CARRIER']))

In [266]:
scalingDF = dfCARRIER[['DISTANCE', 'HDAYS']].astype('float') # Numerical features
categDF = dfCARRIER[['MONTH', 'DAY_OF_MONTH', 'ORIGIN_AIRPORT_ID', 
                    'DEST_AIRPORT_ID', 'ARR_HOUR', 'DEP_HOUR', 
                    'UNIQUE_CARRIER', 'DAY_OF_WEEK']] # Categorical features
y_final = dfCARRIER['ARR_DELAY'].values
y_final.mean()

3.5364078229061753

In [267]:
# Binarisation
if SPARSE:
    # Tout doit être numérique
    # Ensuite on transforme
    from sklearn.preprocessing import OneHotEncoder
    encoder = OneHotEncoder(sparse=True) # Create encoder object
    categDF_encoded = encoder.fit_transform(categDF) # Can't convert this to dense array: too large!
    # Sparse concaténation
    from scipy import sparse 
    scalingDF_sparse = sparse.csr_matrix(scalingDF)
    x_final = sparse.hstack((scalingDF_sparse, categDF_encoded))
else:
    # Comme ça on garde la maitrise des colonnes
    categDF_encoded = pd.get_dummies(categDF.astype(np.str))
    print(categDF_encoded.columns)
    # Concaténation
    x_final = pd.concat([scalingDF, categDF_encoded], axis=1)


Index(['MONTH_1', 'MONTH_10', 'MONTH_11', 'MONTH_12', 'MONTH_2', 'MONTH_3',
       'MONTH_4', 'MONTH_5', 'MONTH_6', 'MONTH_7',
       ...
       'UNIQUE_CARRIER_UA', 'UNIQUE_CARRIER_VX', 'UNIQUE_CARRIER_WN',
       'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4',
       'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7'],
      dtype='object', length=732)


In [268]:
x_final

,DISTANCE,HDAYS,MONTH_1,MONTH_10,MONTH_11,MONTH_12,MONTH_2,MONTH_3,MONTH_4,MONTH_5,...,UNIQUE_CARRIER_UA,UNIQUE_CARRIER_VX,UNIQUE_CARRIER_WN,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7
0,986.0,5.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,986.0,6.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,986.0,7.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,986.0,8.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,986.0,8.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,986.0,7.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,986.0,6.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,986.0,5.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,986.0,4.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9,986.0,3.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [269]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final,y_final,test_size = 0.2,random_state = 0)

# Seules les 2 premières colonnes sont numériques
if SPARSE:
    x_train_numerical = x_train[:, 0:2].toarray()
    x_test_numerical = x_test[:, 0:2].toarray()
else:
    x_train_numerical = x_train.iloc[:, 0:2]
    x_test_numerical = x_test.iloc[:, 0:2]


In [270]:
# Création d'un scaler pour les valeurs numériques 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train_numerical) # fit with the training data ONLY

x_train = x_train.copy()
x_test = x_test.copy()

if SPARSE:
    x_train_numerical = sparse.csr_matrix(scaler.transform(x_train_numerical)) # Transform the data and convert to sparse
    x_test_numerical = sparse.csr_matrix(scaler.transform(x_test_numerical))
    x_train[:, 0:2] = x_train_numerical
    x_test[:, 0:2] = x_test_numerical
else:
    x_train_numerical = scaler.transform(x_train_numerical)
    x_test_numerical = scaler.transform(x_test_numerical)
    x_train.loc[:, 0:2] = x_train_numerical
    x_test.loc[:, 0:2] = x_test_numerical


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [271]:
x_train_numerical

array([[ 0.18847923, -0.50474195],
       [-1.02663432,  1.59912414],
       [ 0.01558409, -0.50474195],
       ..., 
       [ 0.66676861, -0.18107024],
       [ 0.22725945,  2.73197511],
       [ 0.01558409, -0.90933158]])

In [272]:
x_train

,DISTANCE,HDAYS,MONTH_1,MONTH_10,MONTH_11,MONTH_12,MONTH_2,MONTH_3,MONTH_4,MONTH_5,...,UNIQUE_CARRIER_UA,UNIQUE_CARRIER_VX,UNIQUE_CARRIER_WN,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7
193007,0.188479,-0.504742,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
398681,-1.026634,1.599124,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
391572,0.015584,-0.504742,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
215517,-0.596820,-0.342906,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
395423,0.829969,-0.423824,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
374734,-1.210840,0.466273,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
373600,0.733018,2.974729,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
221058,0.001042,-0.585660,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
32662,-1.012092,-0.342906,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
302629,-0.323743,0.061684,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


## GO

In [216]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
    
SGD_params = {'alpha': 10.0**-np.arange(1,7)} # Suggested range we try
SGD_model = GridSearchCV(SGDRegressor(random_state = 0, max_iter=5), SGD_params, scoring = 'neg_mean_absolute_error', cv = 5) # Use 5-fold CV 
SGD_model.fit(x_train, y_train) # Fit the model


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=0, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-01,   1.00000e-02,   1.00000e-03,   1.00000e-04,
         1.00000e-05,   1.00000e-06])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [217]:
SGD_model.best_params_

{'alpha': 0.01}

In [218]:
GridSearchCV(cv=5,estimator=SGDRegressor(alpha=0.001, epsilon=0.1, eta0=0.01, fit_intercept=True,
    l1_ratio=0.15, learning_rate='invscaling', loss='squared_loss',
    n_iter=5, penalty='l2', power_t=0.25, random_state=0, shuffle=False,
    verbose=0, warm_start=False),
    fit_params={}, iid=True, n_jobs=1,
    param_grid={'alpha': [  1.00000e-01,   1.00000e-02,   1.00000e-03,   1.00000e-04, 1.00000e-05,   1.00000e-06]},
    pre_dispatch='2*n_jobs', refit=True, 
    scoring='neg_mean_absolute_error', verbose=0)

GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=5, penalty='l2',
       power_t=0.25, random_state=0, shuffle=False, tol=None, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [219]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

y_true, y_pred = y_test, SGD_model.predict(x_test) # Predict on our test set
    
print ('Mean absolute error of SGD regression : MAE = %.3f' % mean_absolute_error(y_true, y_pred))
print ("RMSE : %.3f" % np.sqrt(mean_squared_error(y_true, y_pred)))

Mean absolute error of SGD regression : MAE = 21.039
RMSE : 37.531


In [239]:
from sklearn.linear_model import LinearRegression
clf = GridSearchCV(LinearRegression(), {}, cv=10, refit=True)
clf.fit(x_train, y_train)
y_true, y_pred = y_test, clf.best_estimator_.predict(x_test)
print("Erreur du modèle linéaire : MAE = %.3f" % mean_absolute_error(y_true, y_pred))
print ("RMSE : %.3f" % np.sqrt(mean_squared_error(y_true, y_pred)))

Erreur du modèle linéaire : MAE = 3325480464.550
RMSE : 50260739154.846


In [229]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(l1_ratio=0.7, alpha=0.001)
enet.fit(x_train, y_train)
y_true, y_pred = y_test, enet.predict(x_test)
print("Erreur ElasticNet : MAE = %.3f" % mean_absolute_error(y_true, y_pred))
print ("RMSE : %.3f" % np.sqrt(mean_squared_error(y_true, y_pred)))

Erreur ElasticNet : MAE = 21.077
RMSE : 38.246


In [262]:
from sklearn.linear_model import ElasticNetCV
l1_ratios = [0.45, 0.5, 0.55, 0.6, 0.65]
min_MAE = 100
for r in l1_ratios:
    enetCV = ElasticNetCV(cv=5, l1_ratio=r, verbose=1)
    enetCV.fit(x_train, y_train)
    y_true, y_pred = y_test, enetCV.predict(x_test)
    mae = mean_absolute_error(y_true, y_pred)
    if mae < min_MAE:
        min_MAE = mae
        best_l1 = r
        best_alpha = enetCV.alpha_
        print("ElasticNetCV : MAE = %.3f" % min_MAE)
        print("alpha=", best_alpha)
        print("l1 ratio=", best_l1)


....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.5s finished


ElasticNetCV : MAE = 20.475
alpha= 0.0180261037917
l1 ratio= 0.45


....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.6s finished
..............................................................................................................................................................................................................................................................................................................................................................................................................................................

ElasticNetCV : MAE = 20.470
alpha= 0.0220335432746
l1 ratio= 0.6


....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.4s finished


0.69999999999999996

In [252]:
from sklearn.linear_model import ElasticNet
alphas = np.logspace(-2, -1, 20)
print("alphas=", alphas)
enet = ElasticNet(l1_ratio=0.7)
for alpha in alphas:
    enet.set_params(alpha=alpha)
    enet.fit(x_train, y_train)
    #print(enet.score(x_train, y_train))
    #print(enet.score(x_test, y_test))
    y_true, y_pred = y_test, enet.predict(x_test)
    print('ElasticNet : alpha=%.3f, MAE=%.3f' % (alpha, mean_absolute_error(y_true, y_pred)))


alphas= [ 0.01        0.01128838  0.01274275  0.0143845   0.01623777  0.01832981
  0.02069138  0.02335721  0.02636651  0.02976351  0.03359818  0.0379269
  0.04281332  0.0483293   0.05455595  0.06158482  0.06951928  0.078476
  0.08858668  0.1       ]
ElasticNet : alpha=0.010, MAE=20.621
ElasticNet : alpha=0.011, MAE=20.600
ElasticNet : alpha=0.013, MAE=20.580
ElasticNet : alpha=0.014, MAE=20.560
ElasticNet : alpha=0.016, MAE=20.539
ElasticNet : alpha=0.018, MAE=20.518
ElasticNet : alpha=0.021, MAE=20.498
ElasticNet : alpha=0.023, MAE=20.481
ElasticNet : alpha=0.026, MAE=20.466
ElasticNet : alpha=0.030, MAE=20.452
ElasticNet : alpha=0.034, MAE=20.443
ElasticNet : alpha=0.038, MAE=20.437
ElasticNet : alpha=0.043, MAE=20.434
ElasticNet : alpha=0.048, MAE=20.435
ElasticNet : alpha=0.055, MAE=20.441
ElasticNet : alpha=0.062, MAE=20.449
ElasticNet : alpha=0.070, MAE=20.462
ElasticNet : alpha=0.078, MAE=20.476
ElasticNet : alpha=0.089, MAE=20.494
ElasticNet : alpha=0.100, MAE=20.515


In [222]:
# Méthode naive par la moyenne pour comparer
from sklearn.dummy import DummyRegressor
dum = DummyRegressor(strategy='mean')
# Entraînement
dum.fit(x_train, y_train)
# Evaluate
y_true, y_pred = y_test, dum.predict(x_test)
print ("Méthode naive par la moyenne : MAE = %.3f" % mean_absolute_error(y_true, y_pred))
print ("RMSE : %.3f" % np.sqrt(mean_squared_error(y_true, y_pred)))

Méthode naive par la moyenne : MAE = 20.922
RMSE : 38.256
